In [1]:
import numpy as np
from functools import reduce

import sys
sys.path.append('..')

from tda.simplicial_complex import Trie, SimplicialComplex, boundary

In [2]:
class f2vector():
    """
    class to hold a vector over F2
    represented by non-zero indices
    """
    def __init__(self, nzinds=[]):
        self.nzinds = sorted(nzinds)
        
    def pivot(self):
        """
        index of last nonzero entry
        
        returns -1 if no nonzeros
        """
        if len(self.nzinds) > 0:
            return self.nzinds[-1]
        else:
            return -1
        
    def nnz(self):
        """
        number of nonzeros in vector
        """
        return len(self.nzinds)
        
    def __add__(self, other):
        """
        add two vectors
        """
        # trivial cases
        if other.nnz() == 0:
            return self
        if self.nnz() == 0:
            return other
        
        newnz = []
        
        i = 0
        j = 0
        while i < self.nnz() and j < other.nnz():
            if self.nzinds[i] < other.nzinds[j]:
                newnz.append(self.nzinds[i])
                i += 1
            elif self.nzinds[i] > other.nzinds[j]:
                newnz.append(other.nzinds[j])
                j += 1
            else:
                i += 1
                j += 1
        
        while i < self.nnz():
            newnz.append(self.nzinds[i])
            i += 1
        
        while j < other.nnz():
            newnz.append(other.nzinds[j])
            j += 1
            
        return f2vector(newnz)
    
    def __str__(self):
        return "f2vector: " + str(self.nzinds)
    
    def __getitem__(self, i):
        if i in self.nzinds:
            return 1
        else:
            return 0
        
v = f2vector([0,1])
print(v)
w = f2vector([0,2])
print(w)
print(v + w)
v[0], v[2]

f2vector: [0, 1]
f2vector: [0, 2]
f2vector: [1, 2]


(1, 0)

In [3]:
from functools import reduce

class f2matrix():
    """
    list of list matrix
    """
    def __init__(self, m=0, n=0, cols=None):
        self.m = m
        self.n = n
        if cols is None:
            self.cols = [f2vector() for _ in range(n)]
        else:
            self.cols = cols
            
    def print(self):
        print("{} x {} f2matrix:".format(self.m, self.n))
        for i in range(self.m):
            for j in range(self.n):
                print(self.cols[j][i], end=' ')
            print()
            
    def __getitem__(self, args):
        if isinstance(args, int):
            j = args
            return self.cols[j]
        elif len(args) == 2:
            i, j = args
            return self.cols[j][i]
        else:
            raise AssertionError("Can use more than 2 indices!")
            
    def __setitem__(self, j, data):
        if not isinstance(j, int):
            raise AssertionError("Only supports column assignment!")
        else:
            self.cols[j] = data
        
    def __mul__(self, other):
        """
        product of 2 f2matrices
        """
        if self.n != other.m:
            raise AssertionError("incompatible dimensions!")
        pass
    
    def nnz(self):
        """
        number of non-zeros
        """
        return reduce(lambda x, y: x+y, [c.nnz() for c in self.cols])
        
        
    def is_zero(self):
        return self.nnz() == 0


A = f2matrix(2,2)
A.print()
print(A.nnz(), A.is_zero())
print()

B = f2matrix(3,3,cols=[f2vector([0,1]), f2vector([1,2]), f2vector([0,2])])
B.print()
print(B.nnz(), B.is_zero())
print()


str(B[0]), B[0,0]

2 x 2 f2matrix:
0 0 
0 0 
0 True

3 x 3 f2matrix:
1 0 1 
1 1 0 
0 1 1 
6 False



('f2vector: [0, 1]', 1)

In [4]:
class f2chain_complex():
    def __init__(self, bdry=[]):
        self.bdry = bdry
        #self.check_boundary()
        
    def __getitem__(self, i):
        return self.bdry[i]
    
    def __setitem__(self, i, data):
        while len(self.bdry) < i+1:
            self.bdry.append(f2matrix())
        self.bdry[i] = data
        
    def maxdim(self):
        return len(self.bdry) - 1
        
    def check_boundary(self):
        for i in range(len(self.bdry)-1):
            if not (self.bdry[i]*self.bdry[i+1]).is_zero():
                raise AssertionError("composition of boundary maps must be zero")
        
C = f2chain_complex()
C[0] = A

In [5]:
X = SimplicialComplex()
X.add((0,1,2))

In [6]:
def f2_boundary_vec(spx, Xk1):
    inds = []
    for f in boundary(spx):
        for i, s in enumerate(Xk1):
            if s == f:
                inds.append(i)
                
    return f2vector(sorted(inds))
                

def f2_boundary(X, k):
    """
    k-dimensional boundary matrix of simplicial complex X
    """
    if k == 0:
        n = len(X.simplices(k))
        return f2matrix(0, n)
    else:
        Xk1 = X.simplices(k-1)
        Xk = X.simplices(k)
        m = len(Xk1)
        n = len(Xk)
        cols = []
        for s in Xk:
            cols.append(f2_boundary_vec(s, Xk1))
        return f2matrix(m, n, cols)
            
f2_boundary(X, 0).print()

0 x 3 f2matrix:


In [7]:
X1 = X.simplices(1)
v = f2_boundary_vec((0,1,2), X1)
print(v)

f2vector: [0, 1, 2]


In [8]:
def f2chain_functor(X, k):
    """
    obtain a chain complex from a simplicial complex X up to dimension k
    """
    C = f2chain_complex()
    for i in range(k+1):
        C[i] = f2_boundary(X, i)
        
    return C
    
    
C = f2chain_functor(X,2)
for k in range(C.maxdim() + 1):
    print('dimension ', k)
    C[k].print()
    print()

dimension  0
0 x 3 f2matrix:

dimension  1
3 x 3 f2matrix:
1 1 0 
1 0 1 
0 1 1 

dimension  2
3 x 1 f2matrix:
1 
1 
1 



In [9]:
def reduce(B):
    """
    performs reduction algorithm on matrix B
    """
    p2c = dict() # pivot to column map
    for j in range(B.n):
        while(B[j].nnz() > 0):
            piv = B[j].pivot()
            j2 = p2c.get(piv, -1)
            if j2 == -1:
                p2c[piv] = j
                break
            else:
                B[j] = B[j] + B[j2]
    return B

print("before reduction")
C[1].print()
reduce(C[1])
print("\nafter reduction:")
C[1].print()

before reduction
3 x 3 f2matrix:
1 1 0 
1 0 1 
0 1 1 

after reduction:
3 x 3 f2matrix:
1 1 0 
1 0 0 
0 1 0 
